<a href="https://colab.research.google.com/github/Sifatkhan-1915020/deeplearning-/blob/main/Prompt%20to%20image%20generation%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sentence-transformers-222:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2893282%2F4988409%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T151433Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D25cedea2582627e453581e7a2fa1e8e1e108cca2ce70dcd2f9eea7859a5a1c94df1e8b4c2a06012badf1104075e514bf3db962badef2a4ac958708af6f4f0db10ac4ffa176860ad998d9169df1e4aaa64c095585a6e52206e047592aaf4325da4fb71969875ac9d728a1dc9c209907c438b373994f3c2de8063cad3f9bb418af7bc449c9f7726edc2752b37d4495f634640f9b9ace1c80366bc8f7fba84e290efbfac19d7270b9a26a75c3cda91cda6444ffbc5c83bdd4e79809186f18ce8a21a21a8ccf87bfd1958cd9ed1083365538b7f7caec5bd9d8a86e754a897de1f095ea4ec79688caeb96429963be9c57d7ba280472dbdd91ede4fe30542e865d9dfa,open-clip-wheels:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2908135%2F5011787%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T151433Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3d6181e7494b7d06bad5e5b15c8ebacec3ce304237c03c5007349b5dd414065f7c392c0cdd75f9b2942d9ae663123f5460a7458b806978e9b862027b7525d8428a61b1c36b171aad92252c877a2a6b262398d156ce7e49b3e79e76c94a9859b61276435356cc350669e62b6bcaf78a6b5ee33c8981174018e81ceec9b0716df603d5be8cff1bd93c077b86a9c5c8efe9987ddd4fdb3c68bdce985cae7c861c07a883d2f08e3a75711138bd503c43ae36ac88926e1bfffa9f06297e88d8937f86d5a2827d09a073a61776d7c49b19c4049d28b5a109bc6d2e5f2c026996edcd692317eb4d3973124b23ddb126bcf2a98c9ff5dcb9b92c15cb1df6870cc59cfc38,open-clip-models:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2908357%2F5012196%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T151433Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D325de25c86b89bd17864c7c31613ccf8c29a865a3e3d3cc852c192e86d8ca88367d72acdc96e1ec2ee254e29b1a75f399d78e0ee90c3c5f8dc457329b8d16b1f8a89829cf1e3e21c19bda9307d8d2455cd45a94f3bbda74cceedda6c86a446b72377bd6b3a02e42bc63b2ceff203a319b571d98c0198213ca84976d9c457e6b4c4fc4e1d7290debbff2401d81bd4b3eac48693ee25d8f9d43c32ce1d662e2c13f96fc56e3f39463a593ef3e3c25172788a81ecb07fa26022a7916db5eb5110bf77c6469452c570873736f14d616d5fc23bb5655dc090b818e95a61463477822a41b354347a8636f18d3f55afbeb72923f6719771e25a24672d65929992f44f02,clip-interrogator-wheels-x:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2910036%2F5014911%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T151433Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D911a8cb9e727193f337da212b2993bf9d5848878fd1ff101a578a68659a6bba69c267c0d5e30cb2f431911831763f29f39dc726414a37437041790394ca963a9aa670e50fe15937695af979c5144168d31b4a2bda0a1baf5fd63f53b3788033f5716b11824eaee7e482004abf24860534280b7a3f198dd2b0d4e2f25e996115d39f8061005f02324370636b97da691a6bde47d41f93adf5a8ce1bf192732e27111ceae1ab2c562bb1b84cb1b76cf19d59819e4a080102c4b65c479485c93dc7486b5bf954a382d864bbdda417ce338f0e13e7e7ce0c7265889e0603b4e4566ae7634037276dfd4f69929c176d172795fc0b58b90d8c6a2064cc968fc8b8e511a,clip-interrogator-models-x:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2910127%2F5023272%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T151433Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e9c54cf7af17d5eb91078875fb6861f9056115e9d2c2577a14ed0c0f3e71c1c848797c05765077d21b144926293ca741f19d8e6a9fc2f31912d30b3f548a0dfd2a27d5da6bed0a1d1876a18f911a612abcf57531de0399bf812130e11f5134955d813a3638aa800d165e449604ea82e729bbed1b3d7fb78ee11078baa00ada31182aee07018c013d130166f0e7dbe4babca7098d2b25ba3b6a37ef9a1e8ef05c65afae118aaa6f3af0958e068f92efdcfc8eeb3945961b19b0ced19a0442ec617db209aabbb6063a61515fba0174002581e38fe6d577715d5e088f68885d664c342b38e56e9426db14a6e6e94614ea08529069689e96cdb84a2e09533e8e253,prompt2image:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2896761%2F5090396%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T151433Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3b3f30e04041ba7f24a5bf64bfefdde949e751ae6383cfdf8a5d1885801cd5d8726be075e9dd53774c0631164d846fa0d85b82b9c03023e90f115f4e6e23a1cf9521e91fac705663f5b92f9604ce4843bf94725b85f55d96378308ac1281ac511063504f0f9022f084fa4cd66fdc65cfca97b954be8e8015cfb5425d09fb8349308dcfddd7c009a6dda991a035ed1cb5d654a946594cd6a774ee297dd49deac86b25d1eefa9a9f7801d39e393380d3cbb52caa75f0f582c4341a2e530c5b4e218e55522c76181a651db861ef6fff6aef61fc49fa229016b3a51bc70094cbdf2f840f3fc92bb8799c85961605c1e3ea85077124bae30eb721e1aa6988ffc3da12,saved-vit-model:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3190170%2F5534730%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T151434Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D323032e9c494dceb8ade06350e85d15f32b5c41634f84c00121551808f7df0a72ed0b30ff402ca0b06c1a0a4ce3b8804573af7197d50f3e8e8932ede3f46aa7dd950e72dd4eb9f4a6a4be4ea25eb406703fa1e1249be564fadb00c1a2fe6cb75974ee4fc9c20e5ad2f1c6a8c1e647919f1d477fc1ec0af8982c4241da1a1053287d11751773cd4fd4c076eef2cdea14fb358f9ff5b79e1e832b4716c658f7c9e9e9de3deafb069da92185481c3a67c3d90c49ed43290ccc20fb1c534ee3c11f9043a0fd69f81c3e6b86c4d3241182f044f19b591f75e2640a6f6ec41f5e62ec968087a31b64e43f71e2d8020e234590dd984648b91ff57f5936d8ee29d807a1d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


![image.png](attachment:ecd05abc-43d9-47cc-aa7d-ff382b3268ca.png)

* Your task in the competition is to extract text out of images

* In order to properly generate embeddings of your predicted prompts, it is crucial that you include the [sentence-transformers-2.2.2](https://www.kaggle.com/datasets/inversion/sentence-transformers-222) dataset

* In this example, the [CoСa OpenClip](https://github.com/mlfoundations/open_clip) model is used to extract text.

* The new stable diffusion model (Stable Diffusion 2.0-v) at 768x768 resolution has the same number of parameters in the U-Net as 1.5, but uses OpenCLIP-ViT/H as the text encoder and is trained from scratch. SD 2.0-v is a so-called v-prediction model. Using this knowledge, subsequent models for extracting text from images can be significantly improved.

# Quick Navigation

* [Imports](#section-one)
* [EDA](#s2)
* [Dataset Used](#s3)
* [Import All Dependencies](#s4)
* [Define Configurations](#s5)
* [Define imt2text CoCa OpenCLIP model](#s6)
* [Extract Prompt From Images](#s7)
* [Evaluation](#s8)
* [Submission](#s9)

<a id="section-one"></a>
# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import glob
import math
import random
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import torch

In [ ]:
df_submission = pd.read_csv('/kaggle/input/stable-diffusion-image-to-prompts/sample_submission.csv', index_col='imgId_eId')
df_submission.head()

<a id="s2"></a>
# EDA

In [ ]:
prompts_df = pd.read_csv("../input/stable-diffusion-image-to-prompts/prompts.csv")
prompts_df

In [ ]:
prompts_df.shape

In [ ]:
# Get the length of each prompt
prompt_lengths = prompts_df["prompt"].apply(lambda x: len(x.split()))

# Plot the distribution of prompt lengths
plt.figure(figsize=(10, 6))
sns.histplot(prompt_lengths, kde=True, color="green")
plt.title("Distribution of Prompt Lengths")
plt.xlabel("Prompt Length")
plt.show()

In [ ]:
from wordcloud import WordCloud

# Join all the prompts together into a single string
text = " ".join(prompt for prompt in prompts_df.prompt)

# Generate a word cloud
wordcloud = WordCloud(width=800, height=500, random_state=42, max_font_size=110).generate(text)

# Plot the word cloud
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

Unnecessary but just for observations 👆

In [ ]:
def image_id2path(
    img_id: str,
    folder: str = "stable-diffusion-image-to-prompts"
) -> str:
    return f"../input/{folder}/images/{img_id}.png"

In [ ]:
df_prompts = prompts_df.copy

**Function to show images and prompts provided,** we will be using this quite often

In [ ]:
def show_images_and_prompts(
    df: pd.DataFrame,
    folder: str = "stable-diffusion-image-to-prompts",
    n: int = 10,
) -> None:
    if n == -1:
        n = df.shape[0]
    for ind, row in df[:n].iterrows():
        img_id = row["imgId"]
        prompt = row["prompt"]
        path = image_id2path(img_id, folder)
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if ind % 2 == 0:
            plt.figure(figsize=(16, 8))
            plt.subplot(1, 2, 1)
        else:
            plt.subplot(1, 2, 2)

        plt.imshow(image)
        list_prompt_words = prompt.split()
        if len(prompt) > 100:
            _len = len(list_prompt_words)
            prompt = "{}\n{}\n{}".format(
                " ".join(list_prompt_words[:_len // 3]),
                " ".join(list_prompt_words[_len // 3 : 2 * _len // 3]),
                " ".join(list_prompt_words[2 * _len // 3:]),
            )
        elif len(prompt) > 50:
            _len = len(list_prompt_words)
            prompt = "{}\n{}".format(
                " ".join(list_prompt_words[:_len // 2]),
                " ".join(list_prompt_words[_len // 2:])
            )
        plt.title(prompt, fontsize=14)
        plt.axis("off")

In [ ]:
if df_prompts is not None:
    show_images_and_prompts(prompts_df, n=7)

<a id="s3"></a>

# Dataset Used

I have used a custom dataset in this notebook which can be found here: https://www.kaggle.com/datasets/ihelon/prompt2image

In [ ]:
df_own = pd.read_csv("../input/prompt2image/prompts.csv", dtype={"imgId": "object"})
df_own

In [ ]:
show_images_and_prompts(
    df=df_own,
    folder="prompt2image",
    n=-1,
)

<a id="s4"></a>
# Import All Dependencies

In [ ]:
df_own = pd.read_csv("../input/prompt2image/prompts.csv", dtype={"imgId": "object"})
df_own

In [ ]:
# For later use
ratio_ViT_B_16          = 0.26
ratio_CLIP_Interrogator = 0.15
ratio_OFA               = 0.09

CKPT_DIR = "/kaggle/input/stable-diffusion-data/OFA-large-caption/"
IMAGE_DIR = "/kaggle/input/stable-diffusion-image-to-prompts/images"

BATCH_SIZE = 24

In [ ]:
wheels_path = "/kaggle/input/open-clip-wheels/open_clip_wheels"
open_clip_whl_path = f"{wheels_path}/open_clip_torch-2.14.0-py3-none-any.whl"

!pip install --no-index --find-links $wheels_path $open_clip_whl_path -q

In [ ]:
import open_clip

In [ ]:
sys.path.append('../input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import SentenceTransformer, models

In [ ]:
comp_path = Path('/kaggle/input/stable-diffusion-image-to-prompts/')

<a id="s5"></a>
# Define Configurations

In [ ]:
class CFG:
    device = "cuda"
    seed = 42
    embedding_length = 384
    sentence_model_path = "/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2"
    model_name = "coca_ViT-L-14"
    model_checkpoint_path = "/kaggle/input/open-clip-models/mscoco_finetuned_CoCa-ViT-L-14-laion2B-s13B-b90k.bin"

Build index from images and compare to sample submission index

In [ ]:
images = os.listdir(comp_path / 'images')
imgIds = [i.split('.')[0] for i in images]

eIds = list(range(CFG.embedding_length))

imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, CFG.embedding_length),
        np.tile(range(CFG.embedding_length), len(imgIds)))]

assert sorted(imgId_eId) == sorted(df_submission.index)

# Load the embedding model

In [ ]:
st_model = SentenceTransformer(CFG.sentence_model_path)

<a id="s6"></a>
# Define imt2text CoCa OpenCLIP model

In [ ]:
model = open_clip.create_model(CFG.model_name)
open_clip.load_checkpoint(model, CFG.model_checkpoint_path)

In [ ]:
transform = open_clip.image_transform(
    model.visual.image_size,
    is_train = False,
    mean = getattr(model.visual, 'image_mean', None),
    std = getattr(model.visual, 'image_std', None),
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

<a id="s7"></a>
# Extract promt from images

In [ ]:
prompts = []

images_path = "../input/stable-diffusion-image-to-prompts/images/"
for image_name in images:
    img = Image.open(images_path + image_name).convert("RGB")
    img = transform(img).unsqueeze(0)

    with torch.no_grad(), torch.cuda.amp.autocast():
        generated = model.generate(img.to(device))

    prompts.append(
        open_clip.decode(
            generated[0]
        ).split("<end_of_text>")[0].replace("<start_of_text>", "").rstrip(" .,")
    )

# Encode prompts for submisssion
prompt_embeddings = st_model.encode(prompts).flatten()

<a id="s8"></a>
# Evaluation

In [ ]:
prompts

In [ ]:
prompts[0]

In [ ]:
prompts_df['prompt']

for i in range(len(prompts_df)):
    prompts_df['prompt'][i] = prompts[i]

In [ ]:
dff = prompts_df.copy()

Reodering list for correct visualisation

In [ ]:
dff.loc[0]['prompt'] = prompts_df['prompt'][3]
dff.loc[1]['prompt'] = prompts_df['prompt'][5]
dff.loc[2]['prompt'] = prompts_df['prompt'][1]
dff.loc[3]['prompt'] = prompts_df['prompt'][2]
dff.loc[4]['prompt'] = prompts_df['prompt'][6]
dff.loc[5]['prompt'] = prompts_df['prompt'][4]
dff.loc[6]['prompt'] = prompts_df['prompt'][0]

In [ ]:
if df_prompts is not None:
    show_images_and_prompts(dff, n=7)

As we can see, our model has performed very well in analysing and describing the context of the picture! We will proceed to make a submission now

<a id="s9"></a>
# Submission

In [ ]:
submission = pd.DataFrame(
    index=imgId_eId,
    data=prompt_embeddings,
    columns=['val']
).rename_axis('imgId_eId').reset_index()

submission

In [ ]:
submission.to_csv('submission.csv', index=False)

Reference notebooks:

* https://www.kaggle.com/code/sajit08/stablediffusion-v2
* https://www.kaggle.com/competitions/stable-diffusion-image-to-prompts
* https://www.kaggle.com/code/maverickss26/lb-0-55962-clipinterrogator-ofa-vit/notebook
* https://www.kaggle.com/code/ihelon/image2prompts-eda-ensembling/notebook
* https://www.kaggle.com/code/leonidkulyk/lb-0-42118-laion-s-coca-vit-openclip

**Thank you** for taking the time to read through my notebook! I hope you found it interesting and informative.

If you liked what you read, consider making an upvote, as it means a lot to me and would aspire other people to check on my results as well!